# https://tinyurl.com/y4tn7g7b
## or
# github.com/njgheorghita/ethdenver

# Icebreaker

# The first rule of EthPM

# Only install packages from **TRUSTED** registries.

In [ ]:
# shoutout explorer/directory

![title](multisig/1.png)

# mission: save grandma's arm

# some words

### registry - on-chain, stores packages
- composed of packages

### package - eg `wallet` or `multisig`
- composed of releases

### release (version) - eg `wallet v1.0.0` or `wallet v1.0.1`
- composed of `package_name` - `version` - `uri`

### manifest - json representing a release

# Setup  <--
##### Deploy "SaveGrandma" multisig
##### Package up "SaveGrandma"
##### Deploy new package registry & tie to ENS
##### Publish "SaveGrandma" package
##### Coop: Fund approve tx "SaveGrandma"
##### Dr. : Approve "SaveGrandma"
##### Save Grandma

In [1]:
# web3.py
from web3.auto.infura import w3
from web3.middleware import construct_sign_and_send_raw_middleware
from ens.utils import raw_name_to_hash

# ethdenver
from secret import DR_PK, DR_ADDRESS, COOP_ADDRESS, coop_w3, GMA_ADDRESS, print_balances

# py-ethpm
from ethpm.tools import builder
from ethpm.utils.chains import create_latest_block_uri
from ethpm.backends.ipfs import InfuraIPFSBackend

# eth-utils
from eth_utils import to_canonical_address, to_hex

In [2]:
# Create the tx-signing middleware
signing_middleware = construct_sign_and_send_raw_middleware(DR_PK)

# Add middleware to target w3 instance
w3.middleware_onion.add(signing_middleware)

# Set default account to Address associated with middleware private key
w3.eth.defaultAccount = DR_ADDRESS

# Enable PM api (temporary)
w3.enable_unstable_package_management_api()

![title](multisig/4.png)

In [3]:
w3.pm

In [4]:
# No canonical registry - should be set for best performance
w3.pm.registry

AttributeError: 'PM' object has no attribute 'registry'

In [5]:
# ethpm.explorer
w3.pm.set_registry("snakecharmers.eth")

In [6]:
w3.pm.registry.address

'0x6b5DA3cA4286Baa7fBaf64EEEE1834C7d430B729'

##### Setup
# Deploy "SaveGrandma" multisig  <--
##### Package up "SaveGrandma"
##### Deploy new package registry & tie to ENS
##### Publish "SaveGrandma" package
##### Coop: Fund approve tx "SaveGrandma"
##### Dr. : Approve "SaveGrandma"
##### Save Grandma

![title](multisig/3.png)

##### chain-agnostic

In [7]:
# ethpm.explorer
multisig_pkg = w3.pm.get_package("multisig", "1.0.1")
multisig_pkg

<Package multisig==1.0.1>

In [8]:
w3.pm.get_release_data("multisig", "1.0.1")

('multisig', '1.0.1', 'ipfs://QmdFykVygYNmGrLM6VUQGf9K2r2zjzeanoY32nh7c2NAwu')

In [9]:
multisig_pkg.uri

'ipfs://QmdFykVygYNmGrLM6VUQGf9K2r2zjzeanoY32nh7c2NAwu'

In [10]:
multisig_factory = multisig_pkg.get_contract_factory("MultiSigWallet") 

In [11]:
# tx_hash = multisig_factory.constructor([DR_ADDRESS, COOP_ADDRESS], 2).transact({"gasPrice": w3.toWei("15", "gwei")})
backup_tx_hash = "0x049750b8015384482b3359041da187a7b2074ddf2f78cdc0d299a77f3b94ca2e"
tx_receipt = w3.eth.waitForTransactionReceipt(backup_tx_hash)
tx_receipt

##### Setup
##### Deploy "SaveGrandma" multisig
# Package up "SaveGrandma"  <--
##### Deploy new package registry & tie to ENS
##### Publish "SaveGrandma" package
##### Coop: Fund approve tx "SaveGrandma"
##### Dr. : Approve "SaveGrandma"
##### Save Grandma

![title](multisig/5.png)

In [13]:
# s/o dev tool & contract alias
new_manifest = builder.build(
    multisig_pkg.manifest,
    builder.version("1.0.2"),
    builder.deployment(
        block_uri = create_latest_block_uri(w3, from_blocks_ago=0),
        contract_instance = 'SaveGrandma',
        contract_type = 'MultiSigWallet',
        address = to_canonical_address(tx_receipt.contractAddress),
        transaction = to_hex(tx_receipt.transactionHash),
        block = to_hex(tx_receipt.blockHash),
    ),
    builder.validate(),
    builder.pin_to_ipfs(backend=InfuraIPFSBackend(), prettify=False),
)
# Check it out on explorer
new_manifest

[{'Name': 'tmpvfdqdru9',
  'Hash': 'QmWcKmHDxcE9oRHskRC9BjN2Y8Rs4Neff7q8suXSaaYrEh',
  'Size': '104747'}]

In [14]:
save_grandma_ipfs_uri = f"ipfs://{new_manifest[0]['Hash']}"
save_grandma_ipfs_uri

'ipfs://QmWcKmHDxcE9oRHskRC9BjN2Y8Rs4Neff7q8suXSaaYrEh'

In [15]:
# grab ENS pkg from snakecharmers.eth
ens_pkg = w3.pm.get_package("ens", "1.0.2")

##### Setup
##### Deploy "SaveGrandma" multisig
##### Package up "SaveGrandma" 
# Deploy new package registry & tie to ENS  <--
##### Publish "SaveGrandma" package
##### Coop: Fund approve tx "SaveGrandma"
##### Dr. : Approve "SaveGrandma"
##### Save Grandma

In [17]:
# w3.pm.deploy_and_set_registry()
w3.pm.set_registry(to_canonical_address('0x8f3c73ba93fb291687c47fca9648d0d10035ec41'))

In [18]:
print("Registry address: ", w3.pm.registry.address)
print("Registry owner:   ", w3.pm.registry.owner())

Registry address:  0x8f3C73BA93FB291687c47FCA9648d0D10035EC41
Registry owner:    0x9182902397B57a8c611D764D4DCD24BA951B4319


![title](multisig/7.png)

In [19]:
# Deployments filtered by w3
resolver = ens_pkg.deployments.get_instance("PublicResolver")

In [20]:
domain = raw_name_to_hash("drmantistoboggan.eth")

In [21]:
resolver.functions.addr(domain).call()

'0x097ddf430E020aEED8dF7e7DF49B41211F3B2494'

In [22]:
txhash_ens = resolver.functions.setAddr(domain, w3.pm.registry.address).transact({'gasPrice': w3.toWei("20 o", "gwei")})
w3.eth.waitForTransactionReceipt(txhash_ens)
# verify on explorer

AttributeDict({'blockHash': HexBytes('0x6e9104ee2cf4542a2c89f8ac26579fcc84ebad59b0aa8a7236894ba01c98d7c7'),
 'blockNumber': 7224253,
 'contractAddress': None,
 'cumulativeGasUsed': 6092040,
 'from': '0x9182902397b57a8c611d764d4dcd24ba951b4319',
 'gasUsed': 34533,
 'logs': [AttributeDict({'address': '0x5FfC014343cd971B7eb70732021E26C35B744cc4',
   'blockHash': HexBytes('0x6e9104ee2cf4542a2c89f8ac26579fcc84ebad59b0aa8a7236894ba01c98d7c7'),
   'blockNumber': 7224253,
   'data': '0x0000000000000000000000008f3c73ba93fb291687c47fca9648d0d10035ec41',
   'logIndex': 66,
   'removed': False,
   'topics': [HexBytes('0x52d7d861f09ab3d26239d492e8968629f95e9e318cf0b73bfddc441522a15fd2'),
    HexBytes('0x0b9c570f29e1c7ee77c056a75649b71f5c61b513125c50582656ec3075e407f1')],
   'transactionHash': HexBytes('0x7181bd9156277aa0475b2d1dbe4be57c7a7dcf0bcb5b24ed6d931b8868cf620e'),
   'transactionIndex': 88})],
 'logsBloom': HexBytes('0x0000000000000000000000000000010000000000000000000000001000000000002000000

##### Setup
##### Deploy "SaveGrandma" multisig
##### Package up "SaveGrandma" 
##### Deploy new package registry & tie to ENS
# Publish "SaveGrandma" package  <--
##### (Coop) Fund & approve tx "SaveGrandma"
##### (Dr.) Approve "SaveGrandma"
##### Save Grandma

In [23]:
# Returns release id
w3.pm.release_package("multisig", "1.0.2", save_grandma_ipfs_uri)

b'\xa1\x04\x08\x19\t\xdd\x1d\xe5\x8eT\xc2dZ\x08\x0bU\xce\xd1\x13+N\xe8\xf6i\xb1\x13+b)\xc02\x9a'

In [24]:
# ethpm.explorer
w3.pm.get_release_data("multisig", "1.0.2")

('multisig', '1.0.2', 'ipfs://QmWcKmHDxcE9oRHskRC9BjN2Y8Rs4Neff7q8suXSaaYrEh')

##### Setup
##### Deploy "SaveGrandma" multisig
##### Package up "SaveGrandma" 
##### Deploy new package registry  & tie to ENS
##### Publish "SaveGrandma" package 
# (Coop) Fund & approve tx "SaveGrandma"  <--
##### (Dr.) Approve "SaveGrandma"
##### Save Grandma

In [25]:
# coop_w3 already setup with auto-signing
coop_w3.pm.set_registry('drmantistoboggan.eth')

In [26]:
coop_w3.pm.registry.address

'0x8f3C73BA93FB291687c47FCA9648d0D10035EC41'

In [27]:
coop_multisig_pkg = coop_w3.pm.get_package("multisig", "1.0.2")

In [28]:
save_grandma = coop_multisig_pkg.deployments.get_instance("SaveGrandma")

In [29]:
print_balances(coop_w3, save_grandma.address)

Grandma:  0 ETH
Multisig: 0 ETH
Coop:     0.096548601597170201 ETH


In [30]:
# Fund multisig

# BUG THIS Wasn't going with 0.1 eth
tx1_hash = save_grandma.fallback.transact({'value': w3.toWei("0.01", "ether")})
tx1_receipt = coop_w3.eth.waitForTransactionReceipt(tx1_hash)

In [31]:
print_balances(coop_w3, save_grandma.address)

Grandma:  0 ETH
Multisig: 0.01 ETH
Coop:     0.086368489597170201 ETH


In [32]:
# Submit tx proposal
# auto confirms tx
tx2_hash = save_grandma.functions.submitTransaction(GMA_ADDRESS, coop_w3.toWei("0.005", "ether"), b'0x').transact()
tx2_receipt = coop_w3.eth.waitForTransactionReceipt(tx2_hash)

##### Setup
##### Deploy "SaveGrandma" multisig
##### Package up "SaveGrandma" 
##### Deploy new package registry & tie to ENS
##### Publish "SaveGrandma" package 
##### (Coop) Fund & approve tx "SaveGrandma" 
# (Dr.) Approve "SaveGrandma"  <--
##### Save Grandma

In [33]:
w3.pm.set_registry("drmantistoboggan.eth")
multisig_pkg = w3.pm.get_package("multisig", "1.0.2")

In [34]:
drs_save_grandma = multisig_pkg.deployments.get_instance("SaveGrandma")

In [36]:
dr_tx = drs_save_grandma.functions.confirmTransaction(0).transact()
w3.eth.waitForTransactionReceipt(dr_tx)

NameError: name 'dr_tx' is not defined

In [37]:
print_balances(coop_w3, save_grandma.address)

Grandma:  0.005 ETH
Multisig: 0.005 ETH
Coop:     0.085387285597170201 ETH


# Grandma has a sweet new arm!

# recap
- deployed and packaged up a multisig instance
- deployed and tied to ENS a new package registry
- published a package
- used the package to give grandma a dope arm

# EthPM

### for developers

### for protocols

### for consultants / auditors / generally impressive and inspiring people

# shoutouts

# gitter.im/ethpm   - @njgheorghita

# links
- https://tinyurl.com/y4tn7g7b
- github.com/njgheorghita/ethdenver


# We who cut mere stones must always be envisioning cathedrals.
##### - medieval quarry workers creed